In [1]:
#### Available if you don't have geo-coding available
#install geopy to do geocoding
#pip install geopy

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import geopy as geo
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.extra.rate_limiter import RateLimiter


# pandas tricks for better display
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

%matplotlib inline

### Hospital Data Loading From Medicare Costs Reporting

#### Hospital Data: Medicare Cost Reporting:

 List of all the hospitals in the US who submit reimbursement (Note this is for Address information for trauma centers, also has medicare ID to link to other data sets):

<b> Download: Hospital Provider Cost Report - Centers for Medicare & Medicaid Services Data (cms.gov)
https://data.cms.gov/provider-compliance/cost-report/hospital-provider-cost-report/data

<b> Dictionary: Hospital Provider Cost Report Data Dictionary - Centers for Medicare & Medicaid Services Data (cms.gov)
https://data.cms.gov/resources/hospital-provider-cost-report-data-dictionary


##### Data Columns to Maintain:
'Provider CCN':  Reformated to Medicare Provider Number a common identifier across data sets
'Hospital Name','Street Address','City','State Code': Name and address fields

'Zip Code': Reformatted to be 5 digit zip text field

'Rural Versus Urban': Suggestion: Make 1/0 Binary

'CCN Facility Type':   https://data.cms.gov/sites/default/files/2019-12/CostReport_CCN_Acronyms_2015_Final_Oct2019.pdf
    Will retain only short term acute and critical access hosptial types
Suggestion:  Focus on STH and CAH, critical access
    
'Provider Type', The number listed best corresponds with the type of services provided. 1 = General Short Term, 2 = General Long Term, 3 = Cancer, 4 = Psychiatric, 5 = Rehabilitation, 6 = Religious NonMedical Health Care Institution, 7 = Children, 8 = Alcohol and Drug, 9 = Other. 
Suggestion:  Focus on 'Type 1'
 
'Type of Control': Indicates the type of control or auspices under which the hospital is conducted as indicated: 1 = Voluntary NonprofitChurch, 2 = Voluntary Nonprofit-Other, 3 = ProprietaryIndividual, 4 = Proprietary-Corporation, 5 = ProprietaryPartnership, 6 = Proprietary-Other, 7 = Governmental-Federal, 8 = Governmental-City-County, 9 = Governmental-County, 10 = Governmental-State, 11 = Governmental-Hospital District, 12 = Governmental-City, 13 = Governmental-Other
Suggestion:  Group by: Govt, Private, Non-Profit
   
'Number of Beds': 
    
'Allowable DSH Percentage':  Lower income patient percentage
    
'Net Patient Revenue'
    
'Net Income'



In [3]:
#Read in hospital data
df_hosp = pd.read_csv('Hospital_Cost_Report_2019.csv')


In [4]:
#select columns of interest
df_hosp = df_hosp.loc[:,['Provider CCN', 'Hospital Name', 'Street Address', 'City', 
                         'State Code','Zip Code', 'Rural Versus Urban', 'CCN Facility Type', 
                         'Provider Type', 'Type of Control', 'Number of Beds', 
                         'Allowable DSH Percentage', 'Net Patient Revenue', 'Net Income']]

#format columns for better data processing
cols_new = df_hosp.columns.str.replace(' ','_').str.lower()
df_hosp.columns = cols_new

#Filter only shortterm acure and critical access hospitals
#Critical access hospitals are facilities with 25 beds or fewer that
#serve primarily rural populations
####DISABLED##### FOR FUTURE USE
#df_hosp = df_hosp[(df_hosp.ccn_facility_type == 'STH') | (df_hosp.ccn_facility_type == 'CAH')]

#rename select columns for consistency and ease of use
nameDict={"provider_ccn":"medicare_prov_num","allowable_dsh_percentage":"dsh_perc",
          "number_of_beds":"beds",'rural_versus_urban':'rur_v_urb'}
df_hosp.rename(columns=nameDict,inplace=True)

#rename select columns for consistency and ease of use
nameDict={"provider_ccn":"medicare_prov_num","allowable_dsh_percentage":"dsh_perc",
          "number_of_beds":"beds",'rural_versus_urban':'rur_v_urb',
         'zip_code':'zip_code_hosp'}
df_hosp.rename(columns=nameDict,inplace=True)

#Standardize zip codes to 5 digit
df_hosp['zip_code_hosp'] = [z if len(z)==5 else z[:5] for z in df_hosp.zip_code_hosp]

#Standardize medicare provider numbers to 6 numbers
df_hosp.medicare_prov_num = df_hosp.medicare_prov_num.astype({'medicare_prov_num':'str'})
df_hosp['medicare_prov_num'] = df_hosp.medicare_prov_num.apply(lambda x: '0' + x if len(x) < 6 else x) #add first 0 back

In [5]:
df_hosp

,medicare_prov_num,hospital_name,street_address,city,state_code,zip_code_hosp,rur_v_urb,ccn_facility_type,provider_type,type_of_control,beds,dsh_perc,net_patient_revenue,net_income
0,040018,BAPTIST HEALTH MEDICAL CENTER - VB,EAST MAIN & SOUTH 20TH STREETS,VAN BUREN,AR,72956,U,STH,1,2,39.0,0.1200,2925510.0,153102.0
1,100044,MARTIN MEDICAL CENTER,200 HOSPITAL AVENUE,STUART,FL,34995,U,STH,1,2,521.0,0.0451,128193757.0,-10629050.0
2,450090,NORTH TEXAS MEDICAL CENTER,1900 HOSPITAL BOULEVARD,GAINSVILLE,TX,76240,R,STH,1,11,36.0,NaN,6034594.0,4580833.0
3,040055,BAPTIST HEALTH MEDICAL CENTER - FS,1001 TOWSON AVENUE,FORT SMITH,AR,72901,U,STH,1,2,320.0,0.1253,35860701.0,-883883.0
4,201302,LINCOLNHEALTH,6 ST. ANDREWS LANE,BOOTHBAY HARBOR,ME,04538,R,CAH,1,2,25.0,NaN,21394222.0,-517955.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6113,181316,MARY BRECKINRIDGE HOSPITAL,130 KATE IRELAND DRIVE,HYDEN,KY,41749,R,CAH,2,2,25.0,NaN,16176608.0,5056590.0
6114,363037,OHIOHEALTH REHABILITATION HOSPITAL,1087 DENNISON AVE 4TH FLOOR,COLUMBUS,OH,43201,U,RH,3,4,74.0,NaN,34110570.0,10890491.0
6115,364007,TWIN VALLEY RPH - COLUMBUS CAMPUS,2200 WEST BROAD STREET,COLUMBUS,OH,43223,U,PH,4,10,228.0,NaN,NaN,-60645644.0
6116,390330,ST. LUKES HOSPITAL - MONROE CAMPUS,100 ST. LUKES LANE,STROUDBURG,PA,18360,U,STH,5,2,98.0,0.0483,156728603.0,21950385.0


### Hospital Medicare Patient Catchment Area

This data set provides a list of patients from zip codes for each of the Medicare IDs from the above data sets.   We could calculate a weighted average distance traveled for patients to each hospital/trauma center using this information.   We could see how average distance to hospital for patients influence KPIs

Hospital Service Area - Centers for Medicare & Medicaid Services Data (cms.gov)
https://data.cms.gov/provider-summary-by-type-of-service/medicare-inpatient-hospitals/hospital-service-area

In [6]:
#load data
df_catchment = pd.read_csv('Hospital_Service_Area_2020.csv')

#format columns for better data processing
cols_new = df_catchment.columns.str.replace(' ','_').str.lower()
df_catchment.columns = cols_new

#rename select columns for consistency and ease of use
nameDict={'zip_cd_of_residence':'zip_code_patient'}
df_catchment.rename(columns=nameDict,inplace=True)

#replace the * with NaN and then Drop NaN values
df_catchment = df_catchment.replace('*',np.NaN).dropna()

#define data types for columns
df_catchment = df_catchment.astype({'medicare_prov_num':'str','zip_code_patient':'str',
                                    'total_days_of_care':'int','total_charges':'int',
                                    'total_cases':'int'}) 
#reformat zip code to be 5-digit
df_catchment.zip_code_patient = df_catchment.zip_code_patient.apply(lambda x: '0' + x if len(x) < 5 else x) #add first 0 back


C:\Users\erinr\AppData\Local\Temp\ipykernel_24480\4270926302.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_catchment = pd.read_csv('Hospital_Service_Area_2020.csv')


In [7]:
df_catchment.head()

,medicare_prov_num,zip_code_patient,total_days_of_care,total_charges,total_cases
106,010001,32420,126,1610103,19
107,010001,32421,67,923209,13
108,010001,32423,169,1854896,32
110,010001,32425,777,9056309,148
111,010001,32426,165,1205519,27


### Create a new data set that includes hospital zip code for geo-distance analysis

In [8]:
#Merge df_hosp and df_catahcment for hospital zip
dist_df = df_catchment.merge(df_hosp,on='medicare_prov_num')
dist_df = dist_df.loc[:,['medicare_prov_num','zip_code_patient','total_days_of_care',
                         'total_cases','hospital_name','zip_code_hosp','state_code']]

dist_df

,medicare_prov_num,zip_code_patient,total_days_of_care,total_cases,hospital_name,zip_code_hosp,state_code
0,010001,32420,126,19,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL
1,010001,32421,67,13,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL
2,010001,32423,169,32,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL
3,010001,32425,777,148,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL
4,010001,32426,165,27,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL
...,...,...,...,...,...,...,...
136681,673071,77450,921,72,ENCOMPASS HEALTH REHABILITATION HOSP,77494,TX
136682,673071,77474,403,29,ENCOMPASS HEALTH REHABILITATION HOSP,77494,TX
136683,673071,77493,324,27,ENCOMPASS HEALTH REHABILITATION HOSP,77494,TX
136684,673071,77494,1043,80,ENCOMPASS HEALTH REHABILITATION HOSP,77494,TX


In [9]:
#import lists of zips with lat/lon information
df_zip = pd.read_csv('Zip_to_lat_lon.csv')

#format data types
df_zip = df_zip.astype({'ZIP':'str','LAT':'float','LNG':'float'}) 

#reformat zip code to be 5-digit
df_zip.ZIP = df_zip.ZIP.apply(lambda x: '0' + x if len(x) < 5 else x) #add first 0 back to both three and four digit
df_zip.ZIP = df_zip.ZIP.apply(lambda x: '0' + x if len(x) < 5 else x) #second pass for three digits that need a fifth

#rename colums for merging and applying df_zip lat and lon to patient zip
df_zip.columns = ['zip_code_patient','patient_lat','patient_lon']
dist_df = dist_df.merge(df_zip,on='zip_code_patient')

#rename colums for merging and applying df_zip lat and lon to hospital zip
df_zip.columns = ['zip_code_hosp','hosp_lat','hosp_lon']
dist_df = dist_df.merge(df_zip,on='zip_code_hosp')


In [10]:
dist_df

,medicare_prov_num,zip_code_patient,total_days_of_care,total_cases,hospital_name,zip_code_hosp,state_code,patient_lat,patient_lon,hosp_lat,hosp_lon
0,010001,32420,126,19,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL,30.632365,-85.384570,31.140065,-85.398289
1,010001,32421,67,13,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL,30.524992,-85.172630,31.140065,-85.398289
2,010001,32423,169,32,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL,30.943777,-85.048909,31.140065,-85.398289
3,010001,32425,777,148,SOUTHEAST HEALTH MEDICAL CENTER,36301,AL,30.853184,-85.720492,31.140065,-85.398289
4,013030,32425,204,18,ENCOMPASS HEALTH REHABILITATION HOSP,36301,AL,30.853184,-85.720492,31.140065,-85.398289
...,...,...,...,...,...,...,...,...,...,...,...
118655,650003,96928,209,24,GUAM REGIONAL MEDICAL CITY,96929,GU,13.384537,144.661298,13.565099,144.876240
118656,650003,96929,3655,388,GUAM REGIONAL MEDICAL CITY,96929,GU,13.565099,144.876240,13.565099,144.876240
118657,650003,96932,1597,198,GUAM REGIONAL MEDICAL CITY,96929,GU,13.475451,144.741733,13.565099,144.876240
118658,650003,96950,385,22,GUAM REGIONAL MEDICAL CITY,96929,GU,15.186892,145.754437,13.565099,144.876240


In [11]:

#create columns with tuples of lat lon for both hospital and patient
dist_df['patient_lat_lon'] = list(zip(dist_df.patient_lat,dist_df.patient_lon))
dist_df['hosp_lat_lon'] = list(zip(dist_df.hosp_lat,dist_df.hosp_lon))

# use geodesic function to calculate the distance in miles from the nearest hosptial
dist_df['distance'] = [geodesic(x, y).miles  for x,y in zip(dist_df.patient_lat_lon,dist_df.hosp_lat_lon)]

###  Zip Census Data for SDOH Measures

Data available to download from:
https://data.census.gov/cedsci/table?g=0100000US%248600000


In [12]:
#Load censuse demographic info
df_cen = pd.read_csv('census_demo_data.csv')

# Keep only certain comlumns of analysis:
cols_to_retain = ['NAME','S0601_C01_001E','S0601_C01_010E','S0601_C01_022E','S0601_C01_036E',
                  'S0601_C01_037E','S0601_C01_047E','S0601_C01_049E']

new_col_names = ['name','total_pop','median_age_yrs','per_white_non_hisp','per_college_deg_25plus',
                 'per_grad_deg_25plus','med_inc_15plus_12mo','per_below_poverty']

df_cen = df_cen.loc[:,cols_to_retain]

df_cen.columns = new_col_names

#Drop first row which contained meta data on measures
df_cen = df_cen.iloc[1:,:]

#create a column for zip code, name for future linking to patient zip
df_cen['zip_code_patient']=df_cen.name.str[-5:]

#drop NA values
df_cen = df_cen.dropna()

#replace trace values with 0's
df_cen = df_cen.replace('-','0')

df_cen = df_cen.replace('**','0')

#replace irregular values
df_cen = df_cen.replace('2,500-','2500')
df_cen = df_cen.replace('250,000+','250000')

#format to integers and floats now that characters are removed
type= {'name':'str','total_pop':'int', 'median_age_yrs':'float','per_white_non_hisp':'float',
       'per_college_deg_25plus':'float','per_grad_deg_25plus':'float','med_inc_15plus_12mo':'int',
       'per_below_poverty':'float','zip_code_patient':'str'}
df_cen= df_cen.astype(type)

#combine percentage of college and graduate degrees
df_cen['per_college_grad_deg_25_plus'] =df_cen['per_college_deg_25plus']+df_cen['per_grad_deg_25plus']
df_cen.pop('per_grad_deg_25plus')
df_cen.pop('per_college_deg_25plus')

C:\Users\erinr\AppData\Local\Temp\ipykernel_24480\2539414011.py:2: DtypeWarning: Columns (2,3,4,5,90,91,92,93,106,107,108,109,126,127,128,129,150,151,152,153,190,191,192,193,214,215,216,217) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cen = pd.read_csv('census_demo_data.csv')


132       20.2
133       25.0
134       13.5
135       16.6
136       23.4
         ...  
33116    100.0
33117      8.7
33118      5.4
33119      0.0
33120     11.5
Name: per_college_deg_25plus, Length: 32989, dtype: float64

### Case Weighting Metrics by Hospital, Aggregating Across Zips

Combine Zip Level Demographic Data with Patient Catchment Location Data

Create weighted average dataframe for each medicare provider number

In [13]:
new_dist_df = dist_df.merge(df_cen,on='zip_code_patient')

In [14]:
#for those columns that we are going to weighted average, 
#multiple the values by the total cases by MPN in that zip code
#then, when we sum the numbers by MPN, we'll divide by the total 
#case population to get the weighted average values

#Chose to weight by new cases since we are looking at emergencies vs. longer term care
#other option would have been to weight by total case days metric

new_dist_df['c_distance'] = new_dist_df['total_cases']*new_dist_df['distance']
new_dist_df['c_median_age'] = new_dist_df['total_cases']*new_dist_df['median_age_yrs']
new_dist_df['c_per_white_non_hisp'] = new_dist_df['total_cases']*new_dist_df['per_white_non_hisp']
new_dist_df['c_med_inc_15plus_12mo'] = new_dist_df['total_cases']*new_dist_df['med_inc_15plus_12mo']
new_dist_df['c_per_below_poverty'] = new_dist_df['total_cases']*new_dist_df['per_below_poverty']
new_dist_df['c_per_college_grad_deg_25_plus'] = new_dist_df['total_cases']*new_dist_df['per_college_grad_deg_25_plus']

In [15]:
#create a data frame that summarizes by MPN
sum_mpn = new_dist_df.pivot_table(index='medicare_prov_num',aggfunc='sum')

#retain only the summary columns
sum_columns=['total_cases','c_distance','c_per_college_grad_deg_25_plus','c_median_age','c_per_white_non_hisp','c_med_inc_15plus_12mo','c_per_below_poverty']
sum_mpn = sum_mpn.loc[:,sum_columns]

C:\Users\erinr\AppData\Local\Temp\ipykernel_24480\4063273176.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_mpn = new_dist_df.pivot_table(index='medicare_prov_num',aggfunc='sum')


In [16]:
#case weight the whole data frame
sum_mpn['cw_distance'] = sum_mpn['c_distance']/sum_mpn['total_cases']
sum_mpn['cw_median_age'] = sum_mpn['c_median_age']/sum_mpn['total_cases']
sum_mpn['cw_per_white_non_hisp'] = sum_mpn['c_per_white_non_hisp']/sum_mpn['total_cases']
sum_mpn['cw_med_inc_15plus_12mo'] = sum_mpn['c_med_inc_15plus_12mo']/sum_mpn['total_cases']
sum_mpn['cw_per_below_poverty'] = sum_mpn['c_per_below_poverty']/sum_mpn['total_cases']
sum_mpn['cw_per_college_grad_deg_25_plus'] = sum_mpn['c_per_college_grad_deg_25_plus']/sum_mpn['total_cases']

# only retain the case weighted values
retain_cols = ['cw_distance','cw_median_age','cw_per_white_non_hisp','cw_med_inc_15plus_12mo',
               'cw_per_below_poverty','cw_per_college_grad_deg_25_plus']

sum_mpn_cw = sum_mpn.loc[:,retain_cols]


#add back the total cases metric before the weighting
sum_mpn_cw['total_cases'] = sum_mpn['total_cases']

In [34]:
sum_mpn_cw

,cw_distance,cw_median_age,cw_per_white_non_hisp,cw_med_inc_15plus_12mo,cw_per_below_poverty,cw_per_college_grad_deg_25_plus,total_cases
medicare_prov_num,,,,,,,
010001,21.476986,42.131482,66.383539,24655.392916,19.612537,17.160008,7849
010005,11.134472,39.520137,80.882204,25209.858882,19.438089,18.474118,3203
010006,13.615921,41.053058,82.300416,25677.575464,15.142540,20.848969,5771
010008,3.923427,42.194656,71.340458,23881.358779,10.209160,23.436641,131
010011,14.007741,39.699132,63.734706,29736.764233,15.592522,21.461692,6218
...,...,...,...,...,...,...,...
673067,20.874744,36.599216,27.682941,26475.960784,19.768039,18.553333,510
673068,7.345930,38.738889,36.488333,45952.150000,6.213889,51.561667,360
673069,16.376355,42.726350,63.952916,38706.380130,6.402592,38.762419,463


### Combine the Summarized, Case Weighted Distances with the Hospital Level Data Set

In [35]:
df_hosp_new = df_hosp.merge(sum_mpn_cw,on='medicare_prov_num')

In [36]:
df_hosp_new.to_csv('Hosp_Pat_Features.csv')

In [50]:
df_hosp_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5055 entries, 0 to 5054
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   medicare_prov_num                5055 non-null   object 
 1   hospital_name                    5055 non-null   object 
 2   street_address                   5052 non-null   object 
 3   city                             5055 non-null   object 
 4   state_code                       5055 non-null   object 
 5   zip_code_hosp                    5055 non-null   object 
 6   rur_v_urb                        5054 non-null   object 
 7   ccn_facility_type                5055 non-null   object 
 8   provider_type                    5055 non-null   int64  
 9   type_of_control                  5055 non-null   int64  
 10  beds                             5052 non-null   float64
 11  dsh_perc                         2504 non-null   float64
 12  net_patient_revenue 

In [49]:
len(df_hosp_new.medicare_prov_num.unique())

4962

In [54]:
pt_sum = df_hosp_new.pivot_table(index='provider_type',aggfunc = 'count',columns=['ccn_facility_type'], 
                                 values = 'medicare_prov_num', margins='True', margins_name = 'Total')
pt_sum

ccn_facility_type,CAH,CH,LTCH,ORD,PH,RH,STH,Total
provider_type,,,,,,,,
1,400.0,NaN,64.0,6.0,13.0,152.0,1918.0,2553
2,752.0,4.0,201.0,NaN,291.0,30.0,241.0,1519
3,16.0,NaN,NaN,NaN,1.0,1.0,17.0,35
4,8.0,NaN,1.0,NaN,2.0,NaN,1.0,12
5,95.0,NaN,NaN,NaN,NaN,102.0,739.0,936
Total,1271.0,4.0,266.0,6.0,307.0,285.0,2916.0,5055


In [ ]:
#Markdown
#'CCN Facility Type':   https://data.cms.gov/sites/default/files/2019-12/CostReport_CCN_Acronyms_2015_Final_Oct2019.pdf
#    Will retain only short term acute and critical access hosptial types
#Suggestion:  Focus on STH and CAH, critical access
ccn_dict = {'CAH': 'Critical Access Hospital', 'CH': 'Children Hospital', 'LTCH':'Long Term Care Hospital', 
            'ORD': 'Hospitals in ORD demo project', 'PH': 'Pychiatric Hospital', 'RH': 'Rehabilitation Hospital', 
            'STH': 'Short Term Acute'}
 
prov_type_dict = {1: 'General Short Term', 2: 'General Long Term', 3: 'Cancer, 4 = Psychiatric, 5 = Rehabilitation,
6 = Religious NonMedical Health Care Institution, 7 = Children, 8 = Alcohol and Drug, 9 = Other. 
Suggestion:  Focus on 'Type 1'


In [68]:
df_hosp_new[df_hosp_new.provider_type == 1].head(10)

,medicare_prov_num,hospital_name,street_address,city,state_code,zip_code_hosp,rur_v_urb,ccn_facility_type,provider_type,type_of_control,beds,dsh_perc,net_patient_revenue,net_income,cw_distance,cw_median_age,cw_per_white_non_hisp,cw_med_inc_15plus_12mo,cw_per_below_poverty,cw_per_college_grad_deg_25_plus,total_cases
0,040018,BAPTIST HEALTH MEDICAL CENTER - VB,EAST MAIN & SOUTH 20TH STREETS,VAN BUREN,AR,72956,U,STH,1,2,39.0,0.1200,2925510.0,153102.0,1.715106,37.180303,81.821212,25482.212121,16.475758,19.706061,132
1,040018,BAPTIST HEALTH MEDICAL CENTER - VB,211 CRAWFORD MEMORIAL DRIVE,VAN BUREN,AR,72956,U,STH,1,2,44.0,0.1200,15858635.0,-2381345.0,1.715106,37.180303,81.821212,25482.212121,16.475758,19.706061,132
2,450090,NORTH TEXAS MEDICAL CENTER,1900 HOSPITAL BOULEVARD,GAINSVILLE,TX,76240,R,STH,1,11,36.0,NaN,6034594.0,4580833.0,2.518397,39.369231,73.416255,32046.296081,14.037300,21.339042,1378
4,040055,BAPTIST HEALTH MEDICAL CENTER - FS,1001 TOWSON AVENUE,FORT SMITH,AR,72901,U,STH,1,2,320.0,0.1253,35860701.0,-883883.0,14.377479,38.926981,71.569950,24660.572972,18.592515,17.707109,12772
5,040055,BAPTIST HEALTH MEDICAL CENTER - FS,1001 TOWSON AVENUE,FORT SMITH,AR,72901,U,STH,1,2,280.0,0.1469,219738586.0,-12159654.0,14.377479,38.926981,71.569950,24660.572972,18.592515,17.707109,12772
6,201302,LINCOLNHEALTH,6 ST. ANDREWS LANE,BOOTHBAY HARBOR,ME,04538,R,CAH,1,2,25.0,NaN,21394222.0,-517955.0,13.146359,51.041766,94.406961,32698.312394,10.761121,36.438031,1178
7,201302,LINCOLNHEALTH,6 ST. ANDREWS LANE,BOOTHBAY HARBOR,ME,04538,R,CAH,1,2,25.0,NaN,68130714.0,2939627.0,13.146359,51.041766,94.406961,32698.312394,10.761121,36.438031,1178
8,201312,WALDO COUNTY GENERAL HOSPITAL,118 NORTHPORT AVENUE,BELFAST,ME,04915,R,CAH,1,2,25.0,NaN,25168895.0,-2222528.0,5.382591,48.904321,94.009671,30922.304527,11.771605,32.780453,972
9,201312,WALDO COUNTY GENERAL HOSPITAL,118 NORTHPORT AVENUE,BELFAST,ME,04915,R,CAH,1,2,25.0,NaN,78261474.0,-29830306.0,5.382591,48.904321,94.009671,30922.304527,11.771605,32.780453,972
10,201315,STEPHENS MEMORIAL HOSPITAL - CAH,181 MAIN STREET,NORWAY,ME,04268,R,CAH,1,2,25.0,NaN,18562833.0,230801.0,7.207380,46.311172,95.819780,26000.946886,15.484799,22.393590,1092


In [60]:
pt1 =  df_hosp_new[(df_hosp_new.provider_type == 1)&
                   ((df_hosp_new.ccn_facility_type=='STH')|(df_hosp_new.ccn_facility_type=='CAH'))]
len(pt1),len(pt1.medicare_prov_num.unique())

(2318, 2297)

In [69]:
len(df_hosp_new),len(df_hosp_new.medicare_prov_num.unique())

(5055, 4962)

### Prepare Outcomes Data

Provider data catalogue with KPIs on Each Hospital:  Cost and Quality of Care Data  tracked by CMS
Data Dictionary and Data Download:  Hospitals | Provider Data Catalog (cms.gov)
https://data.cms.gov/provider-data/topics/hospitals

In [21]:
#read in Cost & Quality of Care data that tracks complications and death data
df_comp_death = pd.read_csv('Complications_and_Deaths_Hospital.csv')
#read in Cost & Quality of Care data that tracks timely effective care
df_time_eff = pd.read_csv('Timely_and_Effective_Care_Hospital.csv')

In [24]:
#rerformat comlumn names
cols_new = df_comp_death.columns.str.replace(' ','_').str.lower()
df_comp_death.columns = cols_new

cols_new = df_time_eff.columns.str.replace(' ','_').str.lower()
df_time_eff.columns = cols_new



In [30]:
df_comp_death.measure_name.unique()

array(['Rate of complications for hip/knee replacement patients',
       'Death rate for heart attack patients',
       'Death rate for CABG surgery patients',
       'Death rate for COPD patients',
       'Death rate for heart failure patients',
       'Death rate for pneumonia patients',
       'Death rate for stroke patients', 'Pressure ulcer rate',
       'Death rate among surgical inpatients with serious treatable complications',
       'Iatrogenic pneumothorax rate',
       'In-hospital fall with hip fracture rate',
       'Postoperative hemorrhage or hematoma rate',
       'Postoperative acute kidney injury requiring dialysis rate',
       'Postoperative respiratory failure rate',
       'Perioperative pulmonary embolism or deep vein thrombosis rate',
       'Postoperative sepsis rate', 'Postoperative wound dehiscence rate',
       'Abdominopelvic accidental puncture or laceration rate',
       'CMS Medicare PSI 90: Patient safety and adverse events composite'],
      dtype=obje

In [32]:
df_time_eff.measure_name.unique()

array(['Emergency department volume',
       'Percentage of healthcare personnel who completed COVID-19 primary vaccination series',
       'Healthcare workers given influenza vaccination',
       'Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better',
       'Average (median) time patients spent in the emergency department before leaving from the visit- Psychiatric/Mental Health Patients.  A lower number of minutes is better',
       'Fibrinolytic Therapy Received Within 30 Minutes of ED Arrival',
       'Left before being seen', 'Head CT results',
       'Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients',
       "Improvement in Patient's Visual Function within 90 Days Following Cataract Surgery",
       'Median Time to Transfer to Another Facility for Acute Coronary Intervention',
       'Appropriate care for severe sepsis and septic shock',
      